本文件仅包含一些测试代码

In [12]:
# test config.json load and parse
import json
from vast import VastClient
from loguru import logger

with open("config.json") as j:
    cfg = json.load(j)

print(cfg)

flag = False
url = cfg['url']
api_key = cfg['vast_api_key']
eth_address = cfg['eth_address']
machine_configs = cfg['machine_configs']
client = VastClient(api_key=api_key, url=url)

{'url': 'https://console.vast.ai', 'vast_api_key': 'bdc70ca54bf58108c0d3305b5b260e7828003c216511d28f6ee81f7b69b7a18f', 'eth_address': '0xEfE59673DE05Bce7CF3aae176A78D03Be15dA676', 'machine_configs': [{'gpu_name': 'RTX_4090', 'gpu_nums': 2, 'dph': 0.5, 'cuda_compute': 89, 'machine_nums': 1}]}


In [8]:
# search offers
import time
from requests.exceptions import HTTPError


def search_offers(cfg):
    global machine_nums, flag, eth_address
    try:
        offers = client.search_offers(
            # reliability > 0.98
            search_query=f'num_gpus = {cfg["gpu_nums"]} dph <= {cfg["dph"]} gpu_name = {cfg["gpu_name"]} '
                         'cuda_vers >= 12.0 verified=false external=false rentable=true',
            sort_order='dph-, dlperf'
        )
    except ValueError as e:
        logger.error(f"Search offers error: {e}",)
        # print("sleep 5 seconds")
        # time.sleep(5)
        raise e

    return offers


offers = search_offers(machine_configs[0])
for offer in offers:
    print(f"ID: {offer.id}, MACHINE_ID: {offer.machine_id}, GPU: {offer.gpu_name}, GPU_NUM: {offer.num_gpus}, DPH: {offer.dph_base}, DLPERF: {offer.dlperf}, VERIFY: {offer.verification}")
# test api limit
# limit = 10 # pre 1s request once
# times = 1000 # pre 1000 request limit - 1
# while limit > 0:
#     offers = search_offers(api_key, machine_configs[0])
#     print(offers)
#     print(f"sleep {limit} s, remain {times}")
#     time.sleep(limit)
#     times -= 1
#     if times == 0:
#         times = 1000
#         limit -= 1

ID: 7427144, MACHINE_ID: 15026, GPU: RTX 4090, GPU_NUM: 2, DPH: 0.5, DLPERF: 145.97836, VERIFY: unverified


In [14]:
# create instances
def create_instance(offer, cfg):
    try:
        res = client.create_instance(id=offer.id, image="nvidia/cuda:12.0.1-devel-ubuntu20.04", ssh=True,
                                     env=f"-e ETH_ADDRESS={eth_address} -e CUDA_COMPUTE={cfg['cuda_compute']} -e GPU_NUMS={cfg['gpu_nums']}",
                                     onstart="onstart.sh")
    except HTTPError as e:
        logger.error(f"Http Error: {e}")
        return False
    if not res['success']:
        logger.error(f"Create Instance error: {res}")
        return False
    return True


# test code
print(create_instance(offers[0], machine_configs[0]))

2023-11-05 17:10:17.674 | INFO     | vast.vast_client:create_instance:595 - rend 7427146 res {'success': True, 'new_contract': 7431513}


True


In [23]:
# test search and create
def robIt(cfg):
    limit = cfg['machine_nums']

    offers = search_offers(cfg)
    if not offers:
        print("not available offers, sleep 5 seconds")
        time.sleep(5)
        # continue
    for offer in offers:
        if create_instance(offer, cfg):
            limit -= 1
            print(f"Create instance success {offer.id}")
        else:
            print(f"Create instance failed {offer.id}")

        if limit == 0:
            break


In [31]:
# test mulit process
from threading import Thread

processes = []
logger.info(f"config: {machine_configs}")
for cfg in machine_configs:
    thread = Thread(target=robIt, args=(cfg,))
    logger.info(f"start thread {thread.ident}")
    thread.start()
    processes.append(thread)

# wait thread finish
for process in processes:
    process.join()

print("program done!!!")

{'gpu_name': 'RTX_4090', 'gpu_nums': 2, 'dph': 0.8, 'cuda_compute': 89, 'machine_nums': 1}


KeyboardInterrupt: 

In [1]:
# init do
from db import init_db

init_db()

In [2]:
# test : get owner instances and insert to db
import json
from vast import VastClient
from db import insert

with open("config.json") as j:
    cfg = json.load(j)
flag = False
url=cfg['url']
api_key = cfg['vast_api_key']
eth_address = cfg['eth_address']
machine_configs = cfg['machine_configs']
client = VastClient(api_key, url)
instances = client.get_instances()
for instance in instances:
    instance.__dict__['image_args'] = ','.join(instance.__dict__['image_args'])
    insert(instance.__dict__)

In [3]:
# get ssh url
from db import fetchOne

instance = fetchOne()
print(instance)
ssh_url = client.get_ssh_url(instance.machine_id, 'ssh://')
# ssh_url2 = 'ssh://root@'+instance.ssh_host+':'+instance.ssh_port
print(f"ssh url: {ssh_url}")


Instance(machine_id=7431513, actual_status=15026, bundle_id='running', bw_nvlink=330410966, compute_cap=0.0, cpu_cores=890, cpu_cores_effective=64, cpu_name=21.333312, cpu_ram='AMD Ryzen Threadripper PRO 5975WX 32-Cores', cpu_util=257615, cuda_max_good=9.398915648915649, cur_state=12.2, direct_port_count='running', direct_port_end=332, direct_port_start=-1, disk_bw=65535, disk_name=4754.0, disk_space='nvme', disk_util=10.0, dlperf=0.675000011920929, dlperf_per_dphtotal=146.079644, dph_base=289.74474842975206, dph_total=0.5, driver_version=0.5041666666666667, duration='535.129.03', end_date=891925.2976810932, external='1700075232.0', flops_per_dphtotal=0, geolocation=322.892080661157, gpu_display_active='Netherlands, NL', gpu_frac=0, gpu_lanes=0.333333, gpu_mem_bw=16, gpu_name=3401.3, gpu_ram='RTX 4090', gpu_temp=24564, gpu_util=35.499985, has_avx=96.499954, host_id=1, hosting_type=47381, id=None, image_args='', image_runtype='ssh_proxy', image_uuid='nvidia/cuda:12.0.1-devel-ubuntu20.04

In [7]:
# 监控执行情况
import paramiko, re, datetime, requests
from prettytable import PrettyTable
from collections import defaultdict
from loguru import logger
ssh_priv_file = '/Users/xiaoxiongma/.ssh/vast_id_rsa'

def instance_list():
    """Function to list instances and get SSH information."""
    url = f'https://console.vast.ai/api/v0/instances/?api_key={api_key}'
    headers = {'Accept': 'application/json'}
    response = requests.get(url, headers=headers)
    ssh_info_list = []

    if response.status_code == 200:
        response_json = response.json()

        if 'instances' not in response_json:
            logger.error("'instances' key not found in response. Please check the API documentation for the correct structure.")
            return ssh_info_list
        instances = response_json['instances']

        # Print information about each instance
        logger.info("Your Instances:")
        
        # Additional variables to store the total_dph for running machines        
        total_dph_running_machines = 0
        
        for instance in instances:
            instance_id = instance.get('id', 'N/A')
            gpu_name = instance.get('gpu_name', 'N/A')
            dph_total = instance.get('dph_total', 'N/A')
            ssh_host = instance.get('ssh_host', 'N/A')
            ssh_port = instance.get('ssh_port', 'N/A')
            num_gpus = instance.get('num_gpus', 'N/A')
            label = instance.get('label', 'N/A')
            actual_status = instance.get('actual_status', 'N/A')
            if actual_status.lower() == 'running':
                total_dph_running_machines += float(dph_total)

            logger.info("Instance ID: %s", instance_id)
            logger.info("GPU Name: %s", gpu_name)
            logger.info("Dollars Per Hour (DPH): %s", dph_total)
            logger.info("SSH Command: ssh -p %s root@%s -L 8080:localhost:8080", ssh_port, ssh_host)
            logger.info("Number of GPUs: %s", num_gpus)
            logger.info("Current state: %s", actual_status)
            logger.info("-" * 30)

            ssh_info = {
                'instance_id': instance_id,
                'gpu_name': gpu_name,
                'dph_total': dph_total,
                'ssh_host': ssh_host,
                'ssh_port': ssh_port,
                'num_gpus': num_gpus,
                'actual_status': actual_status,
                'label': label
            }
            ssh_info_list.append(ssh_info)

    else:
        logger.error("Failed to retrieve instances. Status code: %s. Response: %s", response.status_code, response.text)

    return ssh_info_list, total_dph_running_machines

def clean_ansi_codes(input_string):
    ansi_escape = re.compile(r'\x1B[@-_][0-?]*[ -/]*[@-~]', re.IGNORECASE)
    return ansi_escape.sub('', input_string)


def get_log_info(ssh_host, ssh_port, username):
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy)
    
    try:
        key = paramiko.Ed25519Key(filename=ssh_priv_file)
        ssh.connect(ssh_host, ssh_port, username, pkey=key)
        # Execute the command to get the log information
        _, stdout, _ = ssh.exec_command('tail -n 1 /root/XENGPUMiner/miner.log')
        last_line = stdout.read().decode().strip()
        logger.info(f"Raw log line: {last_line}")
        # Clean ANSI codes from the log line
        last_line = clean_ansi_codes(last_line)
        pattern = re.compile(r'Mining:.*\[(?:(\d+):)?(\d+):(\d+)(?:\.\d+)?,.*?(?:Details=(?:(?:super:(\d+)\s)?normal:(\d+)|xuni:(\d+)).*?)?HashRate:(\d+\.\d+).*Difficulty=(\d+)')
        match = pattern.search(last_line)
        if match:
            # Extracting the running time and blocks information
            hours, minutes, seconds, super_blocks, normal_blocks, xuni_blocks, hash_rate, difficulty = match.groups()
            
            hours = int(hours) if hours is not None else 0
            minutes = int(minutes)
            seconds = int(seconds)
            super_blocks = int(super_blocks) if super_blocks is not None else 0
            normal_blocks = int(normal_blocks) if normal_blocks is not None else 0
            xuni_blocks = int(xuni_blocks) if xuni_blocks is not None else 0
            hash_rate = float(hash_rate)
            difficulty = int(difficulty)
    
            return hours, minutes, seconds, super_blocks, normal_blocks, xuni_blocks, hash_rate, difficulty
        else:
            logger.error("Failed to parse the log line: %s", last_line)
            return None, None, None, None, None, None, None, None
        
    except Exception as e:
        logger.error("Failed to connect or retrieve log info: %s", e)
        return None, None, None, None, None, None, None, None
    finally:
        ssh.close()
    
def print_table(data, mean_difficulty, average_dollars_per_normal_block, total_dph_running_machines, usd_per_gpu, hash_rate_per_gpu, hash_rate_per_usd, label, sum_normal_block_per_hour, output_file='table_output.txt'):
    # Define the table and its columns
    table = PrettyTable()
    table.field_names = ["Instance ID", "GPU Name", "GPU's", "USD/h", "USD/GPU", "Instance h/s", "GPU h/s", "XNM Blocks", "Runtime", "Block/h", "h/s/USD", "USD/Block", "Label"]

    
    # Add rows to the table to console
    for row in data:
        table.add_row(row)

    # Get current timestamp
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    # Print the table
    if mean_difficulty is not None:
        print(f"\nTimestamp: {timestamp}, Difficulty: {int(mean_difficulty)}, Total Hash: {total_hash_rate:.2f} h/s, Total DPH: {total_dph_running_machines:.4f}$, Avg_$/Block: {average_dollars_per_normal_block:.4f}$, Total Blocks/h: {sum_normal_block_per_hour:.2f}")
    print(table)

    # Write the table and timestamp to a text file
    with open(output_file, 'a') as f:
        f.write(f"Timestamp: {timestamp}, Difficulty: {int(mean_difficulty)}, Total Hash: {total_hash_rate:.2f} h/s, Total DPH: {total_dph_running_machines:.4f}$, Avg_$/Block: {average_dollars_per_normal_block:.4f}$, Total Blocks/h: {sum_normal_block_per_hour:.2f}\n{table}\n")
    print(f"Table also written to {output_file}\n")
    

# List Instances and Get SSH Information
ssh_info_list, total_dph_running_machines = instance_list()
username = "root"
sort_order = 'ascending'
sort_column_index = 11


# Initialize Data Storage
gpu_hash_rates = defaultdict(list)
dph_values = []
difficulties = []
hash_rates = []
dollars_per_normal_block_values = []
sum_normal_block_per_hour = 0
table_data = []


# Fetch Log Information for Each Instance
for ssh_info in ssh_info_list:
    instance_id = ssh_info['instance_id']
    gpu_name = ssh_info['gpu_name']
    num_gpus = ssh_info['num_gpus']
    actual_status = ssh_info['actual_status']
    label = ssh_info['label'] if ssh_info['label'] is not None else ''       
    dph_total = float(ssh_info['dph_total'])  # Convert DPH to float for calculations
    dph_values.append(dph_total)
    ssh_host = ssh_info['ssh_host']
    ssh_port = ssh_info['ssh_port']

    logger.info("Fetching log info for instance ID: %s", instance_id)
    hours, minutes, seconds, super_blocks, normal_blocks, xuni_blocks, hash_rate, difficulty = get_log_info(ssh_host, ssh_port, username)

    if num_gpus != 'N/A' and dph_total != 'N/A':
        usd_per_gpu = round(dph_total / float(num_gpus), 4)
    else:
        usd_per_gpu = 'N/A'
    if dph_total != 'N/A' and hash_rate is not None:
        hash_rate_per_usd = round(hash_rate / dph_total, 2)
    else:
        hash_rate_per_usd = 'N/A'        
   
    if difficulty is not None and difficulty != 0:
        difficulties.append(difficulty)
    if hash_rate is not None and hash_rate != 0:
        hash_rates.append(hash_rate)        
    
    if normal_blocks is not None and xuni_blocks is not None:
        runtime_hours = hours + minutes / 60 + seconds / 3600
        logger.info("Running Time: %d hours, %d minutes, %d seconds", hours, minutes, seconds)
        logger.info("Normal Blocks: %d", normal_blocks)
        logger.info("HashRate: %.2f", hash_rate)

        # Calculate Block/h, sum it and handle the case when runtime is zero
        if runtime_hours != 0:
            normal_block_per_hour = normal_blocks / runtime_hours
            # Update sum
            sum_normal_block_per_hour += normal_block_per_hour
        else:
            normal_block_per_hour = 0

        # Calculate $/Blocks and handle the case when the number of blocks is zero
        if normal_blocks != 0:
            dollars_per_normal_block = (runtime_hours * dph_total) / normal_blocks
            dollars_per_normal_block_values.append(dollars_per_normal_block)
        else:
            dollars_per_normal_block = 0
        
        hash_rate_per_gpu = hash_rate / float(num_gpus) if num_gpus != 'N/A' and hash_rate is not None else 'N/A'
        
        if hash_rate is not None and hash_rate != 0 and num_gpus != 'N/A':
            hash_rate_per_gpu = hash_rate / float(num_gpus)
            gpu_hash_rates[gpu_name].append(hash_rate_per_gpu)
        else:
            hash_rate_per_gpu = 'N/A'
        
        table_data.append([instance_id, gpu_name, num_gpus, round(dph_total, 4), round(usd_per_gpu, 4), hash_rate, hash_rate_per_gpu, normal_blocks, round(runtime_hours, 2), round(normal_block_per_hour, 2), round(hash_rate_per_usd, 2), round(dollars_per_normal_block, 2), label])        
    else:
        logger.error("Failed to retrieve log information or normal blocks is None for instance ID: %s", instance_id)


    if difficulties:
        mean_difficulty = sum(difficulties) / len(difficulties)
    else:
        logger.info("No valid difficulties were found.")           
    if hash_rates:
        total_hash_rate = sum(hash_rates)
    else:
        logger.info("No valid HashRate were found.")  
    if dph_values:
        total_dph = sum(dph_values)
    else:
        logger.info("No valid DPH values were found.")
    if dollars_per_normal_block_values:
        average_dollars_per_normal_block = sum(dollars_per_normal_block_values) / len(dollars_per_normal_block_values)
    else:
        average_dollars_per_normal_block = None
        logger.info("No valid $/Block values were found.")

# Sort the data by "<column_name>" in asc or desc order
    if not table_data:
        print("Error: table_data is empty!")
    elif sort_column_index < 0 or (table_data and sort_column_index >= len(table_data[0])):
        print("Invalid sort_column_index: {}. Must be between 0 and {}.".format(sort_column_index, len(table_data[0])-1 if table_data else 'N/A'))
    else:
        # Ensure all rows have the same number of columns
        num_columns = len(table_data[0])
        if all(len(row) == num_columns for row in table_data):
            try:
                # Convert the sort column to float if possible for proper numeric sorting
                table_data.sort(key=lambda x: (float(x[sort_column_index]) if x[sort_column_index] not in (None, 'N/A') else float('-inf'), x), 
                                reverse=(sort_order == 'descending'))
            except ValueError:
                # Fallback to string sorting if conversion to float is not possible
                table_data.sort(key=lambda x: (x[sort_column_index] if x[sort_column_index] not in (None, 'N/A') else '', x), 
                                reverse=(sort_order == 'descending'))
        else:
            print("Error: Not all rows have the same number of columns.")
    

# Print the table
print_table(table_data, mean_difficulty, average_dollars_per_normal_block, total_dph_running_machines, usd_per_gpu, hash_rate_per_gpu, hash_rate_per_usd, label, sum_normal_block_per_hour)


2023-11-05 20:35:04.661 | INFO     | __main__:instance_list:24 - Your Instances:
2023-11-05 20:35:04.662 | INFO     | __main__:instance_list:41 - Instance ID: %s
2023-11-05 20:35:04.663 | INFO     | __main__:instance_list:42 - GPU Name: %s
2023-11-05 20:35:04.664 | INFO     | __main__:instance_list:43 - Dollars Per Hour (DPH): %s
2023-11-05 20:35:04.665 | INFO     | __main__:instance_list:44 - SSH Command: ssh -p %s root@%s -L 8080:localhost:8080
2023-11-05 20:35:04.666 | INFO     | __main__:instance_list:45 - Number of GPUs: %s
2023-11-05 20:35:04.667 | INFO     | __main__:instance_list:46 - Current state: %s
2023-11-05 20:35:04.667 | INFO     | __main__:instance_list:47 - ------------------------------
2023-11-05 20:35:04.671 | INFO     | __main__:<module>:163 - Fetching log info for instance ID: %s
Mining: 1 Blocks [05:49, 325.20s/ Blocks, Details=normal:1, Stat=Active:2, HashRate:6885.81h/s, Difficulty=93400]ks/s]
2023-11-05 20:35:09.281 | INFO     | __main__:<module>:182 - Running


Timestamp: 2023-11-05 20:35:09, Difficulty: 93400, Total Hash: 6885.81 h/s, Total DPH: 0.5042$, Avg_$/Block: 0.0489$, Total Blocks/h: 10.32
+-------------+----------+-------+--------+---------+--------------+----------+------------+---------+---------+---------+-----------+-------+
| Instance ID | GPU Name | GPU's | USD/h  | USD/GPU | Instance h/s | GPU h/s  | XNM Blocks | Runtime | Block/h | h/s/USD | USD/Block | Label |
+-------------+----------+-------+--------+---------+--------------+----------+------------+---------+---------+---------+-----------+-------+
|   7431513   | RTX 4090 |   2   | 0.5042 |  0.2521 |   6885.81    | 3442.905 |     1      |   0.1   |  10.32  | 13657.8 |    0.05   |       |
+-------------+----------+-------+--------+---------+--------------+----------+------------+---------+---------+---------+-----------+-------+
Table also written to table_output.txt


> 记录一些原始api的测试情况

## 获取当前用户设备列表
```
url = "https://console.vast.ai/api/v0/instances?api_key=" + api_key

payload = {}
headers = {
    'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(response.text)
```

## 查询符合要求的机器
```
from urllib.parse import quote
import json

q = {
    "disk_space": {"gte": 16},
    "verified": {"eq": "true"},
    "rentable": {"eq": "true"},
    "num_gpus": {"gte": 2, "lte": 2},
    "gpu_option": {"eq": "RTX 3090"},
    "sort_option": {"0": ["dphtotal", "asc"], "1": ["total_flops", "asc"]},
    "order": [["dphtotal", "asc"]],
    "gpu_name": {"eq": "RTX 3090"},
    "allocated_storage": 16,
    "extra_ids": [],
    "type": "ask"
}
url = "https://console.vast.ai/api/v0/bundles?q=" + quote(json.dumps(q))
payload = {}
headers = {
    'Accept': 'application/json'
}

response = requests.request("GET", url, headers=headers, data=payload)

print(json.dumps(json.loads(response.text), indent=4, sort_keys=True))
```

## 创建实例
```
url = "https://console.vast.ai/api/v0/asks/{id}/?api_key=" + api_key

payload = {
    "client_id": "me",
    "image": "nvidia/cuda:12.0.1-devel-ubuntu20.04",
    "runtype": "ssh ssh_direc ssh_proxy",
    "image_login": None,
    "python_utf8": False,
    "lang_utf8": False,
    "use_jupyter_lab": False,
    "jupyter_dir": None,
    "create_from": None,
    "force": False,
    "template_id": 27230
}
headers = {
    'Accept': 'application/json'
}

response = requests.request("PUT", url, headers=headers, data=payload)

print(response.text)
```

In [ ]:
# python 通过ssh远程执行命令
# import paramiko
